# Maintenance use case

In this use-case, we create an AI assistant that helps software developers streamline the debugging process of their applications. The assistant communicates the functionalities and aids users in understanding complex system structures. It's equipped with capabilities to fetch and interpret logs, interact with databases, connect with APIs, and even query vector databases for document retrieval. The aim is to provide real-time, insightful information that supports problem-solving and makes the debugging process faster and more efficient.

In [1]:
import os

import openai
from openai_functools import FunctionsOrchestrator
from openai_functools.utils.conversation import Conversation

from spoof_functions import generate_spoof_logs, generate_spoof_api_response, generate_spoof_log_analytics, generate_spoof_vector_db_response

## Defining our functions

In [2]:
import datetime
from typing import Dict, Any

def retrieve_logs_by_date_and_vm_id(date: str, vm_id: str) -> str:
    """
    Retrieves logs from a specific date and a specific VM.

    :param str date: The date for which to retrieve the logs, formatted as "YYYY-MM-DD".
    :param str vm_id: The ID of the VM for which to retrieve the logs.
    :return: A string representing the logs of that day for the given VM.
    :rtype: str
    """

    return generate_spoof_logs(date, vm_id)


def retrieve_log_analytics_information() -> str:
    """
    Retrieves aggregate log analytics information.

    :return: A string representing the summary of the logs.
    :rtype: str
    """

    return generate_spoof_log_analytics()


def retrieve_api_information(vm_id: str, endpoint: str, params: Dict[str, Any] = None) -> str:
    """
    Queries an API based on the provided endpoint and parameters.

    :param str vm_id: The ID of the VM to query.
    :param str endpoint: The endpoint of the API to query.
    :param Dict[str, Any] params: A dictionary of parameters to include in the API query. Default is None.
    :return: A dictionary representing the response from the API.
    :rtype: Dict[str, Any]
    """
    return generate_spoof_api_response(vm_id, endpoint, params)


def query_vector_db(query: str) -> str:
    """
    Queries a vector database with an embedded string and returns a piece of text based on similarity.

    :param str query: The string to embed and query the vector database with.
    :return: A string representing the closest match in the vector database.
    :rtype: str
    """
    return generate_spoof_vector_db_response(query)


SYSTEM_MESSAGE = """
You are an AI assistant that helps software developers streamline the debugging process of their applications.
Your prime role is to effectively communicate the functionalities and aid users in understanding complex system structures,
making the debugging process faster and more efficient. You're equipped with the capabilities to fetch and interpret logs,
interact with databases, connect with APIs, and even query vector databases for document retrieval, all with an aim to provide real-time,
insightful information that supports problem-solving. As a digital helper, you understand the intricate operations, the underlying relationships,
and how to tap into the right resources at the right time to provide precise debugging assistance.
Remember, your role is not only to assist but also to educate, guide, and empower the developers in their journey to make robust, high-quality software applications.
"""


## Registering our functions with the orchestrator

In [15]:
orchestrator = FunctionsOrchestrator()
orchestrator.register_all([retrieve_logs_by_date_and_vm_id, retrieve_log_analytics_information, retrieve_api_information, query_vector_db])
conversation = Conversation()
conversation.add_message("system", SYSTEM_MESSAGE)

openai.api_key = os.environ["OPENAI_API_KEY"]

from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Helper functions

In [16]:
# Some helper functions for the maintenance app usecase

FULL_LOG = []
def call_openai(conversation: Conversation):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=conversation.conversation_history,
        functions=orchestrator.create_function_descriptions(),
        function_call="auto",
    )

    FULL_LOG.append(response)
    return response

def handle_response(response, conversation, orchestrator):
    # check if the response is a function call, and if so, call the function
    # else, add the response to the conversation history and display it

    if function_call := response.choices[0].message.function_call:
        function_response = orchestrator.call_function(response)
        conversation.add_message(role="function", content="response from function: " + str(function_response), function_name=function_call.name)
        second_response = call_openai(conversation)
        conversation.add_message(role="assistant", content=second_response.choices[0].message.content)
        handle_response(second_response, conversation, orchestrator)
    else:
        conversation.add_message(role="assistant", content=response.choices[0].message.content)
        conversation.display_last_message()

## Assistant example

In [17]:
conversation.add_message("user", "Hi, I'm having trouble with my application.")
response = call_openai(conversation)
handle_response(response, conversation, orchestrator)

assistant: I'm here to help! Please provide some details about the trouble you're experiencing with your application.




In [18]:
conversation.add_message("user", "Please check log analytics for any information on the error. Can you please provide me with the error code and the VM ID?")
response = call_openai(conversation)
handle_response(response, conversation, orchestrator)

assistant: Based on the log analytics information, I found an error with the following details:

VM ID: VM123
Error Code: ERR001

Please let me know if you need any further assistance with this error.




In [19]:
conversation.add_message("user", "Understood.. please check the logs for more details on the error of that date, can you show me the logs?")
response = call_openai(conversation)
handle_response(response, conversation, orchestrator)

assistant: Here are the logs for the error that occurred on VM123 on 2023-02-01:

```
2023-02-01 10:25:01,156 - INFO - main - Starting application
2023-02-01 10:25:01,156 - INFO - main - Loading configuration
2023-02-01 10:25:01,156 - INFO - main - Connecting to database
2023-02-01 10:25:01,156 - INFO - main - Fetching data from database
2023-02-01 10:25:01,156 - INFO - main - Data successfully fetched
2023-02-01 10:25:01,156 - INFO - main - Starting application
2023-02-01 10:25:01,156 - INFO - main - Loading configuration
2023-02-01 10:25:01,156 - WARNING - main - Database connection timeout
2023-02-01 10:25:01,156 - WARNING - main - Low disk space on server
2023-02-01 10:25:01,156 - ERROR - main - Error loading configuration
2023-02-01 10:25:01,156 - ERROR - component_a - Failed to initialize component - Code: ERR001
```

From the logs, it appears that the application started successfully but encountered errors during the configuration loading process. The database connection timed o

In [20]:
conversation.add_message("user", "Can you check if the VM is running properly? Please check the /healthcheck endpoint. Provide the full response.")
response = call_openai(conversation)
handle_response(response, conversation, orchestrator)

assistant: I apologize, but it seems that there was an internal server error encountered when checking the health of the VM with ID VM123 at the /healthcheck endpoint. The response returned a status code of 500 and an error message of "Internal Server Error". The error code associated with this issue is ERR001.

It appears that there is an issue with the VM, which might be the cause of the errors and warnings mentioned in the logs earlier. I recommend investigating the VM's health and resolving any underlying issues to ensure proper functioning of the application.

If you need further assistance or have any additional questions, please let me know.




In [21]:
conversation.add_message("user", "I need further help understanding the error code. Can you check the vector database to see if there is any relevant documentation?")
response = call_openai(conversation)
handle_response(response, conversation, orchestrator)

assistant: Based on the information retrieved from the vector database, the error code ERR001 indicates a critical alert for an Internal Server Error. This error typically originates from a lower-level system on the server, such as a network socket or a database connection.

Here are some immediate recommended actions to address this error:

1. Run a comprehensive health check on all lower-level systems to ensure they are functioning properly.
2. Check the server's logs for any additional error messages or warnings that may provide insights into the root cause.
3. Confirm the stability of network connections, especially those involved in inter-process communication.
4. If you are using a database, ensure that the connections are pooling correctly and that the database is not reaching its maximum connection limit.
5. If this error is recurring, consider profiling the server's operations to identify any potential bottlenecks or recurring issues that may be contributing to the error.

If 